In [2]:
import pandas as pd
import os
import pathlib
import win32com.client as win32

In [3]:
emails = pd.read_excel(r'Bases de Dados\Emails.xlsx')
lojas = pd.read_csv(r'Bases de Dados\Lojas.csv', encoding = 'latin1', sep=';')
vendas = pd.read_excel(r'Bases de Dados\Vendas.xlsx')

In [4]:
vendas = vendas.merge(lojas, on='ID Loja')

,Código Venda,Data,ID Loja,Produto,Quantidade,Valor Unitário,Valor Final,Loja
0,1,2019-01-01,1,Sapato Estampa,1,358,358,Iguatemi Esplanada
1,1,2019-01-01,1,Camiseta,2,180,360,Iguatemi Esplanada
2,1,2019-01-01,1,Sapato Xadrez,1,368,368,Iguatemi Esplanada
3,21,2019-01-02,1,Camisa Gola V Listrado,2,116,232,Iguatemi Esplanada
4,34,2019-01-02,1,Sapato Listrado,1,363,363,Iguatemi Esplanada
...,...,...,...,...,...,...,...,...
100994,69846,2019-12-25,8,Short Estampa,2,96,192,Salvador Shopping
100995,69846,2019-12-25,8,Tênis Estampa,5,256,1280,Salvador Shopping
100996,69850,2019-12-25,8,Calça Estampa,4,177,708,Salvador Shopping
100997,69972,2019-12-26,8,Terno Liso,3,720,2160,Salvador Shopping


In [5]:
dic_lojas = {}
for loja in lojas['Loja']:
    dic_lojas[loja] = vendas.loc[vendas['Loja'] == loja,:]
dia_indic = vendas['Data'].max()

In [6]:
#C:\Users\GAMER\Favorites\Python_programs\Project 1\Backup Arquivos Lojas
caminho = pathlib.Path(r'C:\Users\GAMER\Favorites\Python_programs\Project 1\Backup Arquivos Lojas')
arq_past_backup = caminho.iterdir()
lista_nomes = [arquivo.name for arquivo in arq_past_backup]
for loja in dic_lojas:
    if loja not in lista_nomes:
        nova_loja = caminho / loja
        nova_loja.mkdir()
        
        
    nome_arquivo = f'{dia_indic.month}_{dia_indic.day}_{loja}.xlsx'
    local_arquivo = caminho / loja / nome_arquivo
    dic_lojas[loja].to_excel(local_arquivo)

In [7]:
meta_fat_dia = 1000
meta_fat_ano = 1650000
meta_qtdeprod_dia = 4
meta_qtdeprod_ano = 120
meta_tkmed_dia = 500
meta_tkmed_ano = 500

In [ ]:
for loja in dic_lojas:
    vendas_loja = dic_lojas[loja]
    vendas_loja_dia = vendas_loja.loc[vendas_loja['Data'] == dia_indic, :]

    #Faturamento
    fat_ano = vendas_loja['Valor Final'].sum()
    fat_dia = vendas_loja_dia['Valor Final'].sum()

    #Diversidade de produtos
    qtde_ano = len(vendas_loja['Produto'].unique())
    qtde_dia = len(vendas_loja_dia['Produto'].unique())

    #Ticket médio
    valor_venda_ano = vendas_loja.groupby('Código Venda').sum()
    valor_venda_dia = vendas_loja_dia.groupby('Código Venda').sum()
    tk_med_ano = valor_venda_ano['Valor Final'].mean()
    tk_med_dia = valor_venda_dia['Valor Final'].mean()
    if fat_dia >= meta_fat_dia:
        cor_fat_dia = 'green'
    else:
        cor_fat_dia = 'red' 
    if qtde_dia >= meta_qtdeprod_dia:
        cor_qtde_dia = 'green'
    else:
        cor_qtde_dia = 'red' 
    if tk_med_dia >= meta_tkmed_dia:
        cor_tk_med_dia = 'green'
    else:
        cor_tk_med_dia = 'red'

    if fat_ano >= meta_fat_ano:
        cor_fat_ano = 'green'
    else:
        cor_fat_ano = 'red' 
    if qtde_ano >= meta_qtdeprod_ano:
        cor_qtde_ano = 'green'
    else:
        cor_qtde_ano = 'red' 
    if tk_med_ano >= meta_tkmed_ano:
        cor_tk_med_ano = 'green'
    else:
        cor_tk_med_ano = 'red' 

    outlook = win32.Dispatch('outlook.application')

    nome = email.loc[emails['Loja']==loja,'Gerente'].values[0]
    mail = outlook.CreateItem(0)
    mail.To = email.loc[emails['Loja']==loja,'E-mail'].values[0]
    mail.Subject = f'OnePage Dia {dia_indic.day}/{dia_indic.month} - Loja {loja}'
    mail.HTMLBody = f'''
    <p>Bom dia, {nome}</p>
    <p>O resultado de ontem <strong>({dia_indic.day}/{dia_indic.month})</strong> da Loja {loja} foi:</p>

    <table>
      <tr>
        <th>Indicador</th>
        <th>Valor Dia</th>
        <th>Meta Dia</th>
        <th>Cenário Dia</th>
      </tr>
      <tr>
        <td>Faturamento</td>
        <td style="text-align: center;"> R${fat_dia:.2f} </td>
        <td style="text-align: center;"> R${meta_fat_dia:.2f} </td>
        <td style="text-align: center; font-size: 20px;"><font color="{cor_fat_dia}"> ֍ </font></td>
      </tr>
      <tr>
        <td>Diversidade de Produtos</td>
        <td style="text-align: center;"> {qtde_dia} </td>
        <td style="text-align: center;"> {meta_qtdeprod_dia} </td>
        <td style="text-align: center; font-size: 20px;"><font color="{cor_qtde_dia}"> ֍ </font></td>
      </tr>
      <tr>
        <td>Ticket Médio</td>
        <td style="text-align: center;"> R${tk_med_dia:.2f} </td>
        <td style="text-align: center;"> R${meta_tkmed_dia:.2f} </td>
        <td style="text-align: center; font-size: 20px;"><font color="{cor_tk_med_dia}"> ֍ </font></td>
      </tr>
    </table>

    <table>
      <tr>
        <th>Indicador</th>
        <th>Valor Ano</th>
        <th>Meta Ano</th>
        <th>Cenário Ano</th>
      </tr>
      <tr>
        <td>Faturamento</td>
        <td style="text-align: center;"> R${fat_ano:.2f} </td>
        <td style="text-align: center;"> R${meta_fat_ano:.2f} </td>
        <td style="text-align: center; font-size: 20px;"><font color="{cor_fat_ano}"> ֍ </font></td>
      </tr>
      <tr>
        <td>Diversidade de Produtos</td>
        <td style="text-align: center;"> {qtde_ano} </td>
        <td style="text-align: center;"> {meta_qtdeprod_ano} </td>
        <td style="text-align: center; font-size: 20px;"><font color="{cor_qtde_ano}"> ֍ </font></td>
      </tr>
      <tr>
        <td>Ticket Médio</td>
        <td style="text-align: center;"> R${tk_med_ano:.2f} </td>
        <td style="text-align: center;"> R${meta_tkmed_ano:.2f} </td>
        <td style="text-align: center; font-size: 20px;"><font color="{cor_tk_med_ano}"> ֍ </font></td>
      </tr>
    </table>

    <p>Segue em anexo a planilha com todos os dados ara mais detalhes.</p>
    <p>Qualquer dúvida estou a disposição.</p>
    <p>Att.,</p>
    <p>Yuri</p>
    '''
    attachment = pathlib.Path.cwd() / caminho / loja / f'{dia_indic.month}_{dia_indic.day}_{loja}.xlsx'
    mail.Attachments.Add(str(attachment))

    mail.Send()

In [10]:
fat_lojas = vendas.groupby('Loja')[['Loja', 'Valor Final']].sum()
fat_lojas_ano = fat_lojas.sort_values(by='Valor Final', ascending=False)

nome_arquivo = f'{dia_indic.month}_{dia_indic.day}_Ranking Anual.xlsx'
local_arquivo = r'Backup Arquivos Lojas\{}'.format(nome_arquivo)
fat_lojas_ano.to_excel(local_arquivo)

vendas_dia = vendas.loc[vendas['Data']==dia_indic, :]
fat_lojas_dia = vendas_dia.groupby('Loja')[['Loja', 'Valor Final']].sum()
fat_lojas_dia = fat_lojas_dia.sort_values(by='Valor Final', ascending=False)

nome_arquivo = f'{dia_indic.month}_{dia_indic.day}_Ranking Diário.xlsx'
local_arquivo = r'Backup Arquivos Lojas\{}'.format(nome_arquivo)
fat_lojas_dia.to_excel(local_arquivo)

In [ ]:
outlook = win32.Dispatch('outlook.application')

mail = outlook.CreateItem(0)
mail.To = email.loc[emails['Loja']=='Diretoria','E-mail'].values[0]
mail.Subject = f'Ranking Dia {dia_indic.day}/{dia_indic.month}'
mail.Body = f'''
Bom dia.

Melhor loja do dia em Faturamento: Loja {fat_lojas_dia.index[0]} = R${fat_lojas_dia.iloc[0, 0]:.2f}
Pior loja do dia em Faturamento: Loja {fat_lojas_dia.index[-1]} = R${fat_lojas_dia.iloc[-1, 0]:.2f}

Melhor loja do ano em Faturamento: Loja {fat_lojas_ano.index[0]} = R${fat_lojas_ano.iloc[0, 0]:.2f}
Pior loja do ano em Faturamento: Loja {fat_lojas_ano.index[-1]} = R${fat_lojas_ano.iloc[-1, 0]:.2f} 


Segue em anexo os rankings do ano e do dia de todas as lojas.

Qualquer dúvida estou a disposição.

Att.,
Yuri
'''
attachment = pathlib.Path.cwd() / caminho / f'{dia_indic.month}_{dia_indic.day}_Ranking Diário.xlsx'
mail.Attachments.Add(str(attachment))
attachment = pathlib.Path.cwd() / caminho / f'{dia_indic.month}_{dia_indic.day}_Ranking Anual.xlsx'
mail.Attachments.Add(str(attachment))

mail.Send()